In [178]:
import os
import mlflow
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder



In [179]:
DATADIR = './data'
# Chargement du dataset
try:
    df = pd.read_csv(os.path.join(DATADIR, 'training_dataset.csv'))
    print("Fichier 'dataset.csv' chargé avec succès.")
except FileNotFoundError:
    print("ERREUR: Le fichier 'training_dataset.csv' est introuvable. Veuillez vérifier son emplacement.")
    exit()

df.describe()

Fichier 'dataset.csv' chargé avec succès.


,revenu_estime_mois,historique_credits,score_credit,loyer_mensuel,montant_pret
count,4707.000000,4707.000000,4707.000000,4707.000000,4707.000000
mean,2502.972382,2.543446,570.654769,794.205957,9177.451199
std,1157.100931,1.691198,109.988345,183.799658,10765.963214
min,500.000000,0.000000,300.000000,265.203900,500.000000
25%,1659.000000,1.000000,553.000000,684.719000,500.000000
50%,2470.000000,3.000000,576.000000,793.052000,3727.548198
75%,3274.000000,4.000000,581.000000,904.830000,16198.325704
max,6826.000000,5.000000,848.000000,1307.961000,53192.053509


In [180]:
if 'df_source' not in vars():
    df_source = df.copy()



In [181]:
#
# Suppresion des credit à 500
#

# df = df[df['montant_pret'] > 500]
df.describe()

,revenu_estime_mois,historique_credits,score_credit,loyer_mensuel,montant_pret
count,4707.000000,4707.000000,4707.000000,4707.000000,4707.000000
mean,2502.972382,2.543446,570.654769,794.205957,9177.451199
std,1157.100931,1.691198,109.988345,183.799658,10765.963214
min,500.000000,0.000000,300.000000,265.203900,500.000000
25%,1659.000000,1.000000,553.000000,684.719000,500.000000
50%,2470.000000,3.000000,576.000000,793.052000,3727.548198
75%,3274.000000,4.000000,581.000000,904.830000,16198.325704
max,6826.000000,5.000000,848.000000,1307.961000,53192.053509


In [182]:
#
# Preparation des features et de la target
#
X = df.drop('montant_pret', axis=1)
y = df['montant_pret']


In [183]:
#
# Pré-traitement des données des features
#

features_numeriques = X.select_dtypes(include=np.number).columns.tolist()
features_text = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nColonnes numériques détectées: {features_numeriques}")
print(f"Colonnes textuelles détectées: {features_text}\n")



Colonnes numériques détectées: ['revenu_estime_mois', 'historique_credits', 'score_credit', 'loyer_mensuel']
Colonnes textuelles détectées: ['niveau_etude', 'region', 'smoker']



In [184]:
#
# Creation d'un pipeline sklearn pour gérer les valeurs numérique et textuelle
#

preprocessor = make_column_transformer(
    (StandardScaler(), features_numeriques),
    (OneHotEncoder(handle_unknown='ignore', sparse_output=False), features_text)
)


In [185]:
#
# Split 80% de data de training, 20% de data de test avec un random_state déterminé pour conserver les mêmes séquences.
#

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [186]:
#
# Preparation des données
#

# 'fit_transform' sur les données d'entraînement pour apprendre les règles de transformation
# le fonctionnement normal est :
# - fit sur les données d'entrainements -> analyse les caractèristiques des données pour savoir les transformation et relation entre les données.
# - transform -> applique les regle sur les données déterminé ci dessus sur les données d'entrainement.
# fit_transform() applique les 2 opération dans une seule fonction
X_train_processed = preprocessor.fit_transform(X_train)
# 'transform' sur les données de test pour appliquer les régles fit qui ont été déterminée grace à fit_transform()
# La on ne veux que faire la transformation avec les même règles que celles déterminée avec les règles d'entrainement.
X_test_processed = preprocessor.transform(X_test)

print("shape du dataset :",X_train_processed.shape)

# Affichage des données preprocessed pour voir à quoi ça ressemble.
X_train_processed

shape du dataset : (3765, 19)


array([[ 1.44811466, -0.31829312,  0.1400017 , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.91461309, -0.91037494, -0.16359922, ...,  0.        ,
         1.        ,  0.        ],
       [-0.859061  ,  0.86587051,  0.04800142, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.66140304, -0.91037494, -0.16359922, ...,  0.        ,
         0.        ,  1.        ],
       [-0.33430535, -0.31829312,  0.05720145, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.56565058,  0.2737887 ,  0.09400156, ...,  0.        ,
         1.        ,  0.        ]])

In [187]:
#
# Construction du model séquentiel (regression)
#

nb_features = X_train_processed.shape[1]
print("nb_features du dataset :", nb_features)

model = tf.keras.Sequential([
    # La couche d'entrée doit correspondre au nombre de features après prétraitement
    # Ici 19 features
    tf.keras.Input(shape=(nb_features,), name='input'),
    tf.keras.layers.Dense(64, activation='relu', name='hidden1_64'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden2_32'),
    # La couche de sortie a 1 neurone car on prédit une seule valeur (le montant)
    tf.keras.layers.Dense(1, name='output_1')
])


nb_features du dataset : 19


In [188]:
#
# Compilation du model
#

model.compile(
    optimizer='adam',
    loss='mean_squared_error',  # Erreur quadratique moyenne
    metrics=[
        'mae',
        tf.keras.metrics.R2Score(name='r2_score')
    ]             # Erreur absolue moyenne (plus facile à interpréter)
)


In [189]:
#
# setup mlflow
# lancer mlflow dans une console via mlflow ui
mlflow.autolog()
mlflow.set_tracking_uri(os.getenv('MLFLOW_URI', 'http://localhost:5000'))


2025/07/27 20:15:40 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/07/27 20:15:40 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 1.3.0 <= scikit-learn <= 1.7.0, but the installed version is 1.7.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/07/27 20:15:40 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/27 20:15:40 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for tensorflow: No module named 'tensorflow.keras'


In [190]:
#
# Entrainement
#
epoch = 75
batch_size = 30
with mlflow.start_run(run_name="training_model_1") as mlrun:
#if True: # Vieux tric pou désactiver mlflow sans devoir desindanter
    history = model.fit(
        X_train_processed,
        y_train,
        epochs=epoch,  # Nombre de passages sur l'ensemble des données
        validation_data=(X_test_processed, y_test),
        verbose=1,   # Affiche une barre de progression
        batch_size=batch_size
    )
    print("--- Performance du model ---")
    loss, mae, r2 = model.evaluate(X_test_processed, y_test, verbose=0)
    print(f"Perte (Loss/MSE) sur l'ensemble de test: {loss:.2f}")
    print(f"Erreur Absolue Moyenne (MAE) sur l'ensemble de test: {mae:.2f} €")
    print(f"(Cela signifie qu'en moyenne, les prédictions du modèle s'écartent de {mae:.2f} € de la valeur réelle)")
    print(f"R2 = {r2:.2f}")

    mlflow.log_metric("loss", loss)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)



2025/07/27 20:15:40 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/75
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 208637936.0000 - mae: 9350.7129 - r2_score: -0.7227 - val_loss: 203693456.0000 - val_mae: 9211.2129 - val_r2_score: -0.7136
Epoch 2/75
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - loss: 201684208.0000 - mae: 9304.6836 - r2_score: -0.7525 - val_loss: 197618656.0000 - val_mae: 8896.7520 - val_r2_score: -0.6625
Epoch 3/75
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - loss: 188251296.0000 - mae: 8666.5889 - r2_score: -0.6374 - val_loss: 180124672.0000 - val_mae: 8598.0156 - val_r2_score: -0.5153
Epoch 4/75
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step - loss: 178729184.0000 - mae: 8732.2188 - r2_score: -0.4772 - val_loss: 153594656.0000 - val_mae: 8394.9951 - val_r2_score: -0.2921
Epoch 5/75
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step - loss: 139854448.0000 - mae: 8245.0322 - r2_score: -0.2356 - val_loss: 128296992.0000 - val_mae: 8276.3760 - val_r2_score: -0.0793
Epoch 6/75
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - loss: 118010776.

2025/07/27 20:15:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


--- Performance du model ---
Perte (Loss/MSE) sur l'ensemble de test: 77304064.00
Erreur Absolue Moyenne (MAE) sur l'ensemble de test: 6825.40 €
(Cela signifie qu'en moyenne, les prédictions du modèle s'écartent de 6825.40 € de la valeur réelle)
R2 = 0.35
🏃 View run training_model_1 at: http://localhost:5000/#/experiments/0/runs/91438109032e4f69af2a8a3d73daef94
🧪 View experiment at: http://localhost:5000/#/experiments/0


Sauvegarder

In [191]:
import joblib
joblib.dump(model, os.path.join('ai_models',f'model_epoch-{epoch}_batch-{batch_size}_r2-{r2}.pkl'))

['ai_models/model_epoch-75_batch-30_r2-0.34967857599258423.pkl']

# Completage nouveau model avec des données du modele précédant

In [192]:
#
# Chargement du nouveau dataset
#

try:
    df = pd.read_csv(os.path.join(DATADIR, 'training_dataset-new.csv'))
    print("Fichier 'dataset.csv' chargé avec succès.")
except FileNotFoundError:
    print("ERREUR: Le fichier 'training_dataset-new.csv' est introuvable. Veuillez vérifier son emplacement.")
    exit()

df.describe()


Fichier 'dataset.csv' chargé avec succès.


,nb_enfants,revenu_estime_mois,historique_credits,score_credit,loyer_mensuel,montant_pret
count,4692.000000,4692.000000,4692.000000,4692.000000,4692.000000,4692.000000
mean,1.598892,2508.968670,2.506820,573.216113,784.634788,7903.309318
std,1.397924,1146.548408,1.688411,107.050928,176.661361,10662.526711
min,-4.000000,500.000000,0.000000,300.000000,260.358100,500.000000
25%,0.000000,1674.000000,1.000000,560.000000,679.603000,500.000000
50%,2.000000,2489.500000,3.000000,575.000000,786.823000,500.000000
75%,3.000000,3295.000000,4.000000,582.000000,889.392500,13415.811885
max,7.000000,6228.000000,5.000000,849.000000,1298.335600,60000.000000


In [193]:
#
# Preparation des features et de la target
#
X = df.drop('montant_pret', axis=1)
y = df['montant_pret']


In [194]:
#
# Pré-traitement des données des features
#

features_numeriques = X.select_dtypes(include=np.number).columns.tolist()
features_text = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nColonnes numériques détectées: {features_numeriques}")
print(f"Colonnes textuelles détectées: {features_text}\n")



Colonnes numériques détectées: ['nb_enfants', 'revenu_estime_mois', 'historique_credits', 'score_credit', 'loyer_mensuel']
Colonnes textuelles détectées: ['sport_licence', 'niveau_etude', 'region', 'smoker']



In [195]:
#
# Creation d'un pipeline sklearn pour gérer les valeurs numérique et textuelle
#

preprocessor = make_column_transformer(
    (StandardScaler(), features_numeriques),
    (OneHotEncoder(handle_unknown='ignore', sparse_output=False), features_text)
)


In [196]:
#
# Split 80% de data de training, 20% de data de test avec un random_state déterminé pour conserver les mêmes séquences.
#

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [197]:
#
# Preparation des données
#

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("shape du dataset :",X_train_processed.shape)

# Affichage des données preprocessed pour voir à quoi ça ressemble.
X_train_processed

shape du dataset : (3753, 22)


array([[ 1.00507416,  2.02429831,  1.4810906 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.42183839, -1.74672307, -0.89587456, ...,  0.        ,
         1.        ,  0.        ],
       [-0.42183839, -0.68519882,  1.4810906 , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.13529466,  1.97992312,  0.88684931, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.71853043,  2.14872287,  1.4810906 , ...,  1.        ,
         0.        ,  1.        ],
       [-1.13529466, -1.09501679,  1.4810906 , ...,  0.        ,
         1.        ,  0.        ]])

In [198]:
#
# Construction du model séquentiel (regression)
#

nb_features = X_train_processed.shape[1]
print("nb_features du dataset :", nb_features)

model_new = tf.keras.Sequential([
    # La couche d'entrée doit correspondre au nombre de features après prétraitement
    # Ici 19 features
    tf.keras.Input(shape=(nb_features,), name='input'),
    tf.keras.layers.Dense(64, activation='relu', name='hidden1_64'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden2_32'),
    # La couche de sortie a 1 neurone car on prédit une seule valeur (le montant)
    tf.keras.layers.Dense(1, name='output_1')
])

nb_features du dataset : 22


In [199]:
#
# Copie du model
#

for layer in model.layers:
    try:
        old_layer = model.get_layer(layer.name)
        layer.set_weights(old_layer.get_weights())
        print(f"✅ Poids transférés pour : {layer.name}")
    except ValueError:
        print(f"⛔ Incompatible ou nouvelle couche : {layer.name}")


✅ Poids transférés pour : hidden1_64
✅ Poids transférés pour : hidden2_32
✅ Poids transférés pour : output_1


In [200]:
#
# Compilation du model
#

model_new.compile(
    optimizer='adam',
    loss='mean_squared_error',  # Erreur quadratique moyenne
    metrics=[
        'mae',
        tf.keras.metrics.R2Score(name='r2_score')
    ]             # Erreur absolue moyenne (plus facile à interpréter)
)


In [201]:
#
# Entrainement
#
epoch = 75
batch_size = 100
with mlflow.start_run(run_name="training_model_2") as mlrun:
#if True: # Vieux tric pou désactiver mlflow sans devoir desindanter
    history = model_new.fit(
        X_train_processed,
        y_train,
        epochs=epoch,  # Nombre de passages sur l'ensemble des données
        validation_data=(X_test_processed, y_test),
        verbose=1,   # Affiche une barre de progression
        batch_size=batch_size
    )
    print("--- Performance du model_new ---")
    loss, mae, r2 = model_new.evaluate(X_test_processed, y_test, verbose=0)
    print(f"Perte (Loss/MSE) sur l'ensemble de test: {loss:.2f}")
    print(f"Erreur Absolue Moyenne (MAE) sur l'ensemble de test: {mae:.2f} €")
    print(f"(Cela signifie qu'en moyenne, les prédictions du modèle s'écartent de {mae:.2f} € de la valeur réelle)")
    print(f"R2 = {r2:.2f}")

    mlflow.log_metric("loss", loss)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)


2025/07/27 20:15:52 WARNING mlflow.keras.autologging: Failed to log dataset information to MLflow. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/75
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 169109232.0000 - mae: 7748.7534 - r2_score: -0.5518 - val_loss: 189026368.0000 - val_mae: 8291.1826 - val_r2_score: -0.5715
Epoch 2/75
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 176213360.0000 - mae: 7927.2573 - r2_score: -0.5548 - val_loss: 188773424.0000 - val_mae: 8277.0625 - val_r2_score: -0.5694
Epoch 3/75
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - loss: 168827424.0000 - mae: 7679.3711 - r2_score: -0.5367 - val_loss: 188033152.0000 - val_mae: 8236.7451 - val_r2_score: -0.5632
Epoch 4/75
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 170846864.0000 - mae: 7696.2642 - r2_score: -0.5293 - val_loss: 186404800.0000 - val_mae: 8148.8423 - val_r2_score: -0.5497
Epoch 5/75
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 158794400.0000 - mae: 7359.3169 - r2_score: -0.5135 - val_loss: 183502144.0000 - val_mae: 7990.6821 - val_r2_score: -0.5255
Epoch 6/75
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 165079200.0000 - mae: 7348.757

2025/07/27 20:15:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


--- Performance du model_new ---
Perte (Loss/MSE) sur l'ensemble de test: 65518728.00
Erreur Absolue Moyenne (MAE) sur l'ensemble de test: 5828.28 €
(Cela signifie qu'en moyenne, les prédictions du modèle s'écartent de 5828.28 € de la valeur réelle)
R2 = 0.46
🏃 View run training_model_2 at: http://localhost:5000/#/experiments/0/runs/9102aec87fcc43d1ab881992d87225d4
🧪 View experiment at: http://localhost:5000/#/experiments/0


In [202]:
joblib.dump(model_new, os.path.join('ai_models',f'model_new_epoch-{epoch}_batch-{batch_size}_r2-{r2}.pkl'))

['ai_models/model_new_epoch-75_batch-100_r2-0.45531147718429565.pkl']